In [ ]:
import pandas as pd
import numpy as np
import perf
import metrics

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
%matplotlib inline

In [ ]:
0.217744


In [20]:
! g++ -Wall -std=c++11 -O3 src/*.cpp -o ./bin/sgd

src/main.cpp:65:28: error: no matching constructor for initialization of
      'LinearRegressionResult'
    LinearRegressionResult result(p, n, x, y, ypred, exposure, model.coeffs);
                           ^      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/metrics.h:22:5: note: candidate constructor not viable: no known conversion
      from 'std::vector<float>' to 'float *' for 5th argument
    LinearRegressionResult(int p, int n, uint8_t* x, float* y, float* ypred,
    ^
src/metrics.h:8:7: note: candidate constructor (the implicit copy constructor)
      not viable: requires 1 argument, but 7 were provided
class LinearRegressionResult
      ^
src/metrics.h:8:7: note: candidate constructor (the implicit move constructor)
      not viable: requires 1 argument, but 7 were provided
src/main.cpp:68:5: error: cannot delete expression of type 'std::vector<float>'
    delete[] ypred;
    ^        ~~~~~
2 errors generated.


In [18]:
!time ./bin/sgd

rmse: (0) 0.947487 0
rmse: (100000) 0.216787 0
shuffling 163964
shuffling 387950
shuffling 308680
shuffling 387782
shuffling 655247
shuffling 802066
shuffling 1081325
shuffling 873541
shuffling 167075
shuffling 1214486
shuffling 489486
shuffling 1100115
shuffling 573349
shuffling 709478
shuffling 731055
shuffling 211370
shuffling 551853
shuffling 1366852
shuffling 790905
shuffling 432848
shuffling 1219849
shuffling 404511
shuffling 200546
shuffling 1310301
shuffling 527556
shuffling 1320706
shuffling 737895
shuffling 1274708
shuffling 733569
shuffling 1363971
shuffling 2191
shuffling 139026
shuffling 362367
shuffling 477698
shuffling 778303
shuffling 1253532
shuffling 1029333
shuffling 153009
shuffling 940037
shuffling 1164856
shuffling 1280669
shuffling 446354
shuffling 1345306
shuffling 361064
shuffling 292658
shuffling 1288903
shuffling 874870
shuffling 33925
shuffling 423418
shuffling 362950
shuffling 589956
shuffling 1297615
shuffling 285231
shuffling 761090
shuffling 700699
shuff

In [ ]:
df = pd.read_csv("./data/results.csv")
print('mean:', round(np.sum(df.prediction) / np.sum(df.exposure), 6), round(np.sum(df.target) / np.sum(df.exposure), 6))
print('rmse:', round(perf.root_mean_square_error(df.target, df.prediction, df.exposure), 6))
print('gini:', metrics.gini_emblem_fast(df.target, df.prediction, df.exposure))
print('deviance:', np.log(metrics.poisson_deviance(df.target, df.prediction, df.exposure)))
metrics.plot_lift_curve(df.target, df.prediction, df.exposure)

In [ ]:
0.21607

mean: 0.043815 0.044271
rmse: 0.216067
gini: 0.32862359198864044
deviance: 12.322948734266841